In [1]:
import pandas as pd
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sn
import matplotlib.mlab as mlab
%matplotlib inline

### Описание датасета

В датасетах представлены данные, которые группа ученых из Фрамингема (США) использовала для выявления риска заболевания ишемической болезнью сердца в течение 10 лет.

Демографические данные: 

    sex (male): пол, мужчина (1) или женщина (0) 

    age: возраст 
    
    education: уровень образования (0-4: школа-колледж)
    
Поведенческие данные: 

    currentSmoker: курильщик (1) или нет (0)

    cigsPerDay: количество выкуриваемых сигарет в день (шт.)

Медицинская история:

    BPMeds: принимает ли пациент препараты для регулировки артериального давления (0 - нет, 1 - да)

    prevalentStroke: случался ли у пациента сердечный приступ (0 - нет, 1 - да)

    prevalentHyp: страдает ли пациент гипертонией (0 - нет, 1 - да)

    diabetes: страдает ли пациент диабетом (0 - нет, 1 - да)

Физическое состояние:

    totChol: уровень холестерина

    sysBP: систолическое (верхнее) артериальное давление 

    diaBP: диастолическое (нижнее) артериальное давление 

    BMI: индекс массы тела - масса (кг) / рост^2 (в метрах)

    heartRate: пульс

    glucose: уровень глюкозы

Целевая переменная (на которую авторы строили регрессию):

    TenYearCHD: риск заболевания ишемической болезнью сердца в течение 10 лет

### Построим логистическую модель прогноза риска ишемической болезни сердца в 10 летней перспективе в зависимости от возраста.

In [86]:
Fram_train = pd.read_csv('Fram_train.csv')
Fram_test = pd.read_csv('Fram_test.csv')

In [85]:
Fram_train.head(1)

,Unnamed: 0,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0


In [87]:
Fram_train.dropna(axis=0,inplace=True)
Fram_test.dropna(axis=0,inplace=True)

In [88]:
# разбиваем датафрейм на две части, dfx - параметры, dfy - целевая переменная. 
dfx = Fram_train[['age']]
dfy = Fram_train[['TenYearCHD']]

In [89]:
# используем логистическую регрессию из sklearn. Имя lm - для краткости записи
lm = linear_model.LogisticRegression(solver='liblinear') 
# solver='liblinear' задает алгоритм поиска максимума функции правдоподобия. 

In [102]:
# функция fit обучает регрессию - подбирает коэффициенты
model = lm.fit(dfx, dfy.values.ravel()) 

In [103]:
# выведем коэффициенты модели
model.coef_

array([[0.06177387]])

In [104]:
# выведем коэффициент при константе
model.intercept_

array([-4.86358689])

### К какому классу отнесёт модель первого пациента из тестовой выборки?

In [119]:
Fram_test[:1][['age']]

,age
0,50


In [121]:
model.predict(Fram_test[:1][['age']])

array([0])

### Посмотрим на всех тестовых данных

In [124]:
y_pred = model.predict(Fram_test[['age']])

In [128]:
y_test = Fram_test[['TenYearCHD']]

In [130]:
# смотрим accuracy модели
lm.score(Fram_test[['age']], y_test)

0.8555555555555555

In [131]:
# смотрим confusion matrix - таблицу правильных и неправильных предсказаний
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix 

array([[539,   0],
       [ 91,   0]])

### Обучим логистическую модель классификации по признакам:

- количество сигарет в день,
- наличие/отсутствие диабета,
- уровень холестерина в крови,
- систолическое давление,
- индекс массы тела,
- уровень глюкозы в крови.

In [135]:
dfx1 = Fram_train[['cigsPerDay', 'diabetes', 'totChol', 'sysBP', 'BMI', 'glucose']]
dfy1 = Fram_train[['TenYearCHD']]

In [136]:
model1 = lm.fit(dfx1, dfy1.values.ravel())

In [137]:
y_pred = model1.predict(Fram_test[['cigsPerDay', 'diabetes', 'totChol', 'sysBP', 'BMI', 'glucose']])

In [138]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix 

array([[537,   2],
       [ 87,   4]])

In [143]:
# Количевство пациентов группы риска в тестовой выборке
y_test.sum()

TenYearCHD    91
dtype: int64

In [145]:
# смотрим accuracy модели
model1.score(Fram_test[['cigsPerDay', 'diabetes', 'totChol', 'sysBP', 'BMI', 'glucose']], y_test)

0.8587301587301587